In [1]:
!pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

In [2]:
pip show  dlt

Name: dlt
Version: 1.14.1
Summary: dlt is an open-source python-first scalable data loading library that does not require any backend to run.
Home-page: https://github.com/dlt-hub
Author: 
Author-email: "dltHub Inc." <services@dlthub.com>
License-Expression: Apache-2.0
Location: /opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.9/site-packages
Requires: click, fsspec, gitpython, giturlparse, hexbytes, humanize, jsonpath-ng, orjson, packaging, pathvalidate, pendulum, pluggy, pytz, pyyaml, requests, requirements-parser, rich-argparse, semver, setuptools, simplejson, sqlglot, tenacity, tomlkit, typing-extensions, tzdata
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
import dlt

In [4]:
import requests

In [5]:
@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [6]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [7]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2025-07-18 10:41:46.029771+00:00 and COMPLETED in 5.79 seconds with 4 steps.
Step extract COMPLETED in 0.76 seconds.

Load package 1752835308.069067 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.06 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- zoomcamp_data: 948 row(s)

Load package 1752835308.069067 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 2.94 seconds.
Pipeline zoomcamp_pipeline load step completed in 2.93 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /Users/mlyon/db.qdrant location to store data
Load package 1752835308.069067 is LOADED and contains no failed jobs

Step run COMPLETED in 5.79 seconds.
Pipeline zoomcamp_pipeline load step completed in 2.93 seconds
1 load package(s) were loaded to destination qdrant and into datas